In [ ]:
import pandas as pd
import plotly.express as px
import seaborn as sns
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from sqlalchemy import create_engine
import urllib

In [ ]:
ca_zip_population = pd.read_csv('CA Zip with Lon and Lat.csv').drop(columns=('Unnamed: 0'))

In [ ]:
ca_zip_population

In [ ]:
# ca_zip_population['latitude'] = ''
# ca_zip_population['longitude'] = ''

# for n in range(len(ca_zip_population)):
#     location = geolocator.geocode(ca_zip_population['zip'][n])
#     ca_zip_population['latitude'][n] = location[1][0]
#     ca_zip_population['longitude'][n] = location[1][1]
    
# ca_zip_population.to_csv('CA Zip with Lon and Lat.csv')

In [ ]:
ca_zip_population.dtypes

In [ ]:
# This creates the connection to the California Cannabis files in SQL

params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};"
                                 "SERVER=bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com;"
                                 "DATABASE=ca_cannabis;"
                                 "UID=admin;"
                                 "PWD=N19lrqxnurTUJLJT6GFe")
engine_con = create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))

ca_main = pd.read_sql("SELECT * FROM ca_main", engine_con)
ca_contact = pd.read_sql("SELECT * FROM ca_contact", engine_con)

In [ ]:
# This gives us the count of retailers per zipcode and puts it into a clean df
company_count = pd.DataFrame(ca_contact[ca_contact['license_number'].isin(ca_main[ca_main['license_description'] == 'Retailer']['license_number'])]['contact_zip'].value_counts())
company_count_df = company_count.reset_index().rename(columns={'index': 'zip', 'contact_zip': 'company_count'})

In [ ]:
company_count_df['zip'] = company_count_df['zip'].astype(int)

In [ ]:
ca_zip_population_and_retail_count = pd.merge(ca_zip_population, company_count_df, how='left', on='zip')

In [ ]:
ca_zip_population_and_retail_count['population_per_retailer'] = (ca_zip_population_and_retail_count['population']/ca_zip_population_and_retail_count['company_count']).fillna(0).astype(int)

In [ ]:
ca_zip_population_and_retail_count.head()

In [ ]:
fig = px.scatter_mapbox(ca_zip_population_and_retail_count, lat='latitude', lon='longitude',
                        hover_name='zip', 
                        center = {'lat': 37, 'lon': -120},
                        color='population_per_retailer',
                        size='population',
#                         color_continuous_scale=px.colors.cyclical.IceFire,
                        zoom=5, width=1000, height=1000
               )
fig.update_layout(mapbox_style="open-street-map")
# fig.figure(figsize=(20, 20))
fig.show()

In [ ]:
fig = px.scatter_mapbox(ca_zip_population_and_retail_count, lat='latitude', lon='longitude',
                        hover_name='zip', 
                        center = {'lat': 37, 'lon': -120},
                        color='population_per_retailer',
#                         size='population',
#                         color_continuous_scale=px.colors.cyclical.IceFire,
                        zoom=5, width=1000, height=1000
               )
fig.update_layout(mapbox_style="open-street-map")
# fig.figure(figsize=(20, 20))
fig.show()